## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [124]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [125]:
wine = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv', index_col=0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Recall that we need to check for missing data.

In [126]:
#A summary of missing variables represented as a percentage of the total missing content. 
def missingness_summary(df, print_log=False, sort='ascending'):
  s = df.isnull().sum()*100/df.isnull().count()
    
  if sort.lower() == 'ascending':
    s = s.sort_values(ascending=True)
  elif sort.lower() == 'descending':
    s = s.sort_values(ascending=False)  
  if print_log: 
    print(s)    
  
  return pd.Series(s)

In [127]:
# answer below:
missingness_summary(wine)


fixed.acidity           0.0
volatile.acidity        0.0
citric.acid             0.0
residual.sugar          0.0
chlorides               0.0
free.sulfur.dioxide     0.0
total.sulfur.dioxide    0.0
density                 0.0
pH                      0.0
sulphates               0.0
alcohol                 0.0
quality                 0.0
dtype: float64

In [128]:
wine['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

Convert quality to a binary variable, with the dividing line between 5 and 6.


In [129]:
# answer below
wine['bQuality'] = np.where(wine['quality'] >= 6, 1, 0)
wine.describe()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,bQuality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023,0.534709
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569,0.498950
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000,0.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000,0.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000,1.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000,1.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000,1.000000


Create a train test split with 20% of the data in the test subsample.

In [130]:
y = wine['bQuality']
X = wine.drop(columns=['quality', 'bQuality'])

# answer below:
from sklearn.model_selection import train_test_split, cross_val_score

#Size of the test set.
SIZE = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SIZE) 

Scale only the independent variables using the minmax scaler.

In [131]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# answer below:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) 


Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [132]:
from sklearn import neighbors

# answer below:
knn = neighbors.KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

knn_scaled = neighbors.KNeighborsRegressor(n_neighbors=5)
knn_scaled.fit(X_train_scaled, y_train)

print("Unscaled score: {}, Scaled score: {}".format(knn.score(X_train, y_train),  knn_scaled.score(X_train_scaled, y_train)))

Unscaled score: 0.3882748141154252, Scaled score: 0.5107959597151737


When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [133]:
def weights(dist):
  # if user attempts to classify a point that was zero distance from one
  # or more training points, those training points are weighted as 1.0
  # and the other points as 0.0
  if dist.dtype is np.dtype(object):
    for point_dist_i, point_dist in enumerate(dist):
      # check if point_dist is iterable
      # (ex: RadiusNeighborClassifier.predict may set an element of
      # dist to 1e-6 to represent an 'outlier')
      if hasattr(point_dist, '__contains__') and 0. in point_dist:
        dist[point_dist_i] = point_dist == 0.
      else:
        dist[point_dist_i] = 1. / np.sqrt(point_dist)
  else:
    with np.errstate(divide='ignore'):
      dist = 1. / np.sqrt(dist)
    inf_mask = np.isinf(dist)
    inf_row = np.any(inf_mask, axis=1)
    dist[inf_row] = inf_mask[inf_row]
  return dist
 

In [134]:
# answer below:
knn_w = neighbors.KNeighborsRegressor(n_neighbors=7, weights=weights)
knn_w.fit(X_train_scaled, y_train)

knn_w.score(X_train_scaled, y_train)

1.0

In [135]:
knn_test_w = neighbors.KNeighborsRegressor(n_neighbors=7, weights=weights)
knn_test_w.fit(X_test_scaled, y_test)

knn_test_w.score(X_test_scaled, y_test)

1.0